In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import Trainer, TrainingArguments
!pip install datasets


  Using cached datasets-3.3.2-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.9 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv('/content/sentimentdataset.csv')
df.head()

,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,USA,2023,1,15,12
1,1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,2023,1,15,8
2,2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,USA,2023,1,15,15
3,3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,UK,2023,1,15,18
4,4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,2023,1,15,19


In [ ]:

df['Sentiment'] = df['Sentiment'].map({'positive': 0, 'neutral': 1, 'negative': 2})



In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(df['Text'], df['Sentiment'], test_size=0.2, random_state=42)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Text'],
    df['Sentiment'],
    test_size=0.2,  # 20% for validation
    random_state=42
)

In [ ]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels.values
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels.values
))

In [ ]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']


dhetdjryfry


In [ ]:
# Tokenize the train and validation data
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='tf')
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=512, return_tensors='tf')


In [ ]:
# Map sentiments to integers (if not already done)
train_labels = train_labels.map({'positive': 0, 'neutral': 1, 'negative': 2})
val_labels = val_labels.map({'positive': 0, 'neutral': 1, 'negative': 2})


In [ ]:
import tensorflow as tf

# Convert tokenized data into a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),  # Input data (tokenized)
    train_labels           # Labels (mapped to integers)
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),  # Input data (tokenized)
    val_labels           # Labels (mapped to integers)
))


2nd correction


In [ ]:
# Check if any labels are missing or invalid
print(train_labels.isnull().sum())  # Check for missing values in training labels
print(val_labels.isnull().sum())    # Check for missing values in validation labels


585
147


In [ ]:

# Remove rows with invalid labels
train_dataset_cleaned = train_dataset.filter(lambda x, y: not tf.math.is_nan(y))  # Filter train_dataset based on train_labels
val_dataset_cleaned = val_dataset.filter(lambda x, y: not tf.math.is_nan(y))    # Filter val_dataset based on val_labels

In [ ]:
# Map the sentiment labels to integers
label_map = {'positive': 0, 'neutral': 1, 'negative': 2}

train_labels = train_labels.map(label_map)
val_labels = val_labels.map(label_map)

# Check if the labels have been mapped correctly
print(train_labels.unique())  # Should print [0, 1, 2] or other valid integer labels
print(val_labels.unique())    # Should print [0, 1, 2] or other valid integer labels


[nan]
[nan]


In [ ]:


# Map the sentiment labels to integers, handling missing values
label_map = {'positive': 0, 'neutral': 1, 'negative': 2}

train_labels = train_labels.map(label_map).fillna(-1).astype(int)  # Fill NaN with -1 and convert to int
val_labels = val_labels.map(label_map).fillna(-1).astype(int)      # Fill NaN with -1 and convert to int


In [ ]:
print(train_labels.unique())  # Should print [0, 1, 2] or other valid integer labels
print(val_labels.unique())

[-1]
[-1]


In [ ]:
train_labels = train_labels.astype(int)
val_labels = val_labels.astype(int)


In [ ]:
# Convert tokenized text and labels to TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

# Check the first entry in the dataset to verify the format
for text, label in train_dataset.take(1):
    print(text)  # Should print a dictionary of tokenized inputs (input_ids, attention_mask)
    print(label)  # Should print an integer label (0, 1, or 2)


{'input_ids': <tf.Tensor: shape=(39,), dtype=int32, numpy=
array([  101, 10842,  2006,  1996,  7860,  1997,  1996,  2082,  2095,
        1012,  3110,  1037,  2978, 13394,  2007, 14799,  1012,   102,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(39,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(39,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)>}
tf.Tensor(-1, shape=(), dtype=int64)


In [ ]:
# Train the model
history = model.fit(
    train_dataset.shuffle(1000).batch(8),  # Shuffle and batch the training data
    validation_data=val_dataset.batch(8),  # Batch the validation data
    epochs=3
)


Epoch 1/3


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbas

3rd correction

In [ ]:
# Map the sentiment labels to integers
label_map = {'positive': 0, 'neutral': 1, 'negative': 2}

train_labels = train_labels.map(label_map)
val_labels = val_labels.map(label_map)

# Remove rows with missing labels (NaN values after mapping)

# Remove rows with invalid labels
train_dataset_cleaned = train_dataset.filter(lambda x, y: not tf.math.is_nan(y))  # Filter train_dataset based on train_labels
val_dataset_cleaned = val_dataset.filter(lambda x, y: not tf.math.is_nan(y))

# Convert tokenized text and labels to TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels.astype(int) # Convert labels to int after filtering
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels.astype(int) # Convert labels to int after filtering
))

# Train the model
history = model.fit(
    train_dataset.shuffle(1000).batch(8),
    validation_data=val_dataset.batch(8),
    epochs=3
)

TypeError: in user code:

    File "<ipython-input-68-39b676c91597>", line 10, in None  *
        lambda x, y: not tf.math.is_nan(y)

    TypeError: Value passed to parameter 'x' has DataType int64 not in list of allowed values: bfloat16, float16, float32, float64
